## Task 2.0: Download and load data (using duckDB)

In [8]:
# Download the data from github
! rm -rf SA-takehome
! git clone https://github.com/mcough2/SA-takehome.git
#! pip install duckdb


Cloning into 'SA-takehome'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 57 (delta 10), reused 17 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 1.55 MiB | 3.41 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [3]:
from pathlib import Path
import duckdb

In [1]:
# Connect to DuckDB
con = duckdb.connect('egress.db')

# Load each CSV file into DuckDB table
egress_data = Path('SA-takehome/Sample Egress Data/').rglob('*.csv')

for file in egress_data:
    # Extract the table name from the file name after the '-'
    table_name = file.name.split(' - ')[1].replace('.csv', '')
    
    # Use the table name when creating the table in DuckDB
    con.execute(f"CREATE TABLE {table_name} AS SELECT * FROM read_csv_auto('{file}')")

In [29]:
# Reconnect to DuckDB database
con = duckdb.connect('egress.db')
# Get the list of tables
tables = con.execute("SHOW TABLES").fetchall()
print(tables)
# Expose schema for each table
for table in tables:
    table_name = table[0]
    print(f"Table: {table_name}")
    print(con.execute(f"DESCRIBE {table_name}").fetchdf())
    #print(con.execute(f"SELECT * FROM {table_name} LIMIT 5").fetchdf())
    print("\n")

[('billable_metric',), ('customer',), ('events',), ('invoice',), ('line_item',), ('plan',), ('plan_charge',), ('product',), ('sub_line_item',)]
Table: billable_metric
        column_name column_type null   key default extra
0                id     VARCHAR  YES  None    None  None
1         aggregate     VARCHAR  YES  None    None  None
2    aggregate_keys     VARCHAR  YES  None    None  None
3  environment_type     VARCHAR  YES  None    None  None
4        group_keys     VARCHAR  YES  None    None  None
5              name     VARCHAR  YES  None    None  None
6        created_at   TIMESTAMP  YES  None    None  None
7       archived_at     VARCHAR  YES  None    None  None
8        updated_at   TIMESTAMP  YES  None    None  None


Table: customer
                     column_name column_type null   key default extra
0                             id     VARCHAR  YES  None    None  None
1                           name     VARCHAR  YES  None    None  None
2                 ingest_aliases   

## Task 2.1
**Description**: Using the sample egress data provided, write a query that calculates the total number of images that were generated for each size between March 10th and March 25th (inclusive).

In [5]:
# Converts PGSQL key-value string to JSON 
def convert_kv_to_json(kv_str: str) -> str:
    kv_str = kv_str.replace('=', '":"')
    kv_str = kv_str.replace(', ', '", "')
    kv_str = kv_str.replace('{', '{"')
    kv_str = kv_str.replace('}', '"}')
    return kv_str

# Register the function in DuckDB
#con.remove_function("convert_kv_to_json")
con.create_function("convert_kv_to_json", convert_kv_to_json)

In [6]:
# Get the number of images generated for the period between March 10 and March 25, 2024
# Filter rows down to the period between March 10 and March 25, 2024
# And where event_type is equal to 'image_modeler'
imq_query_events_only = """

WITH event_unpack AS (
    SELECT 
        event_type, 
        customer_id,
        timestamp,
        JSON(convert_kv_to_json(properties)) AS event_properties
    FROM events e
    WHERe timestamp >= '2024-03-10' AND timestamp <= '2024-03-25'
    AND event_type IN ('image_modeler')
    AND properties IS NOT NULL)

    select name,
           customer_id,
           event_properties.image_size,
           SUM(CAST(event_properties.num_images AS INTEGER)) total_images 
    FROM event_unpack e
    LEFT JOIN customer c on e.customer_id = c.id
    WHERE event_properties.image_size IS NOT NULL
    GROUP BY 1, 2, 3
    ORDER BY total_images
"""


con.execute(imq_query_events_only).fetchdf()
# Output final results to CSV including col heads
#con.execute(imq_query_events_only).fetchdf().to_csv('task2_1_image_modeler.csv', index=False)

,name,customer_id,image_size,total_images
0,None,b1_company,"""1024x1024""",1212.0
1,None,b1_company,"""512×512""",1243.0
2,None,b1_company,"""256×256""",1309.0


In [33]:
query2 = """ select name,id from customer
where name = 'B1 Company'"""
con.execute(query2).fetchdf()

,name,id
0,B1 Company,7c5a8bc2-823e-4b11-9f41-6d5e711c2485


## Task 2.1: Notes & Observations
The instructions are ambiguous about what customer or client to use. Perhaps this is because only one customer falls within the timeframe given.

There isn't a proper customer id match between the customer & events tables.

## Task 2.2
**Description**: Recreate the March invoice for customer A1 Company. Write a query that returns a list of all the charges (description, quantity, unit price, total) for the finalized March invoice for customer A1 Company. The screenshot below provides a sample of a February finalized invoice. The query should return the corresponding lines under the CPU Hours and Storage products for March

In [8]:
march_invoice = """
with granular_items as(
SELECT c.name, 
       i.id as invoice_id,
       --i.status as invoice_status,
       --i.plan_name as invoice_plan_name,
       i.issued_at as invoice_issue_date,
       --i.start_timestamp as invoice_start_ts,
       --i.end_timestamp as invoice_end_ts,
       l.name as line_item_name,
       l.total as line_item_total, 
       l.quantity as line_item_quantity,
       l.updated_at as line_item_updated_at,
       sl.*,

    FROM customer c
left join invoice i on c.id = i.customer_id
left join line_item l on i.id = l.invoice_id
left join sub_line_item sl on l.id = sl.line_item_id
where c.name = 'A1 Company'
and i.start_timestamp >= '2024-02-29' and i.end_timestamp <= '2024-03-31'),

agg_1 as (SELECT line_item_name,
       billable_metric_name, 
       sum(quantity) as quantity,
       round(max(subtotal) / 100, 2) as total
       from granular_items
group by 1,2
order by 1,2, 3 asc)

select line_item_name,
        billable_metric_name as Description,
        quantity as Quantity,
        CONCAT('$', round((total / quantity), 2), ' USD') as "Unit Price",
        CONCAT('$', total) as Total
        
 from agg_1
"""
con.execute(march_invoice).fetchdf()


,line_item_name,Description,Quantity,Unit Price,Total
0,CPU Hours,CPU hours (c5.4xlarge),25.00,$0.01 USD,$0.25
1,CPU Hours,CPU hours (c5.8xlarge),32.00,$0.02 USD,$0.58
2,CPU Hours,CPU hours (m5.12xlarge),33.00,$0.25 USD,$8.25
3,CPU Hours,CPU hours (m5.16xlarge),68.00,$0.03 USD,$2.38
4,CPU Hours,CPU hours (m5.4xlarge),42.00,$0.02 USD,$0.92
5,CPU Hours,CPU hours (r5.2xlarge),94.00,$0.22 USD,$20.68
6,CPU Hours,CPU hours (r5.8xlarge),42.00,$0.01 USD,$0.59
7,Storage,Storage GB-Hours (ca-central-1),7575.75,$0.0 USD,$11.36
8,Storage,Storage GB-Hours (us-east-1),6347.25,$0.0 USD,$6.35
9,Storage,Storage GB-Hours (us-east-2),18430.00,$0.0 USD,$20.27


In [10]:
# Write line item results to CSV
con.execute(march_invoice).fetchdf().to_csv('./submissions/task2_2_invoice.csv', index=False)

## Task 2.2
**Description**: Generate a report of billings by Plan for the month of March 2024
*NOTE*: I assumed this to mean for the same company, `A1`.

In [43]:
march_invoice_by_plan = """
SELECT 
    p.name AS plan_name,
    c.name AS customer_name,
    CONCAT('$', ROUND(SUM(i.total/100),2), 'USD') AS total_billed
FROM 
    invoice AS i
JOIN 
    plan AS p ON i.plan_id = p.id
JOIN 
    customer AS c ON i.customer_id = c.id
WHERE 
    i.issued_at BETWEEN '2024-03-01' AND '2024-03-31'
AND c.name = 'A1 Company'
GROUP BY 
    p.name, c.name
ORDER BY 
    total_billed DESC;
"""
con.execute(march_invoice_by_plan).fetchdf()#.to_csv('./submissions/task2_3_by_plan_c1_co.csv', index=False)

,plan_name,customer_name,total_billed
0,Infra SaaS Paygo,A1 Company,$71.63USD
